In [29]:
import re
import codecs
import numpy as np
from numpy import zeros, int8, log
from pyvi import ViTokenizer
from pylab import random

class PLSA:
    def __init__(self, K, maxIteration=30, threshold=10.0, topicWordsNum=5):
        self.K = K  # Số chủ đề
        self.maxIteration = maxIteration  # Số lần lặp tối đa
        self.threshold = threshold  # Ngưỡng dừng
        self.topicWordsNum = topicWordsNum  # Số từ hàng đầu trong mỗi chủ đề
        self.Pz = zeros(self.K)  # Xác suất của các chủ đề
        self.lamda = None  # Ma trận lambda P(d|z)
        self.theta = None  # Ma trận theta P(w|z)
        self.p = None  # Ma trận xác suất trung gian P(z|d,w)

    def initializeParameters(self, N, M, Pz, lamda, theta):
        
        self.Pz = Pz
        self.lamda = lamda
        self.theta = theta

        print("Initialization")
        print("Pz\n", self.Pz)
        print("theta\n", self.theta)
        print("lamda\n", self.lamda)

    def EStep(self, N, M):
        """
        Thực hiện bước E: Tính toán ma trận xác suất P(z|d,w).
        """
        for i in range(0, N):
            for j in range(0, M):
                denominator = np.sum(self.Pz[k] * self.lamda[i,k] * self.theta[j,k] for k in range(self.K))
                if denominator == 0:
                    self.p[i,j,:] = 1.0/self.K
                else:
                    for k in range(self.K):
                        self.p[i,j,k] = (self.Pz[k] * self.lamda[i,k] * self.theta[j,k]) / denominator    


    def MStep(self, N, M, X):
        # Cập nhật theta (P(w|z))
        for k in range(self.K):
            for j in range(M):  # Loop over words (M should be the number of words)
                self.theta[j,k] = np.sum(X[i, j] * self.p[i, j, k] for i in range(N))
            self.theta[:,k] /= np.sum(self.theta[:,k])

        # Cập nhật lambda (P(d|z))
        for k in range(self.K):
            for i in range(N):  # Loop over documents (N should be the number of documents)
                self.lamda[i, k] = np.sum(X[i, j] * self.p[i, j, k] for j in range(M))
            self.lamda[:, k] /= np.sum(self.lamda[:, k])

        # Cập nhật Pz
        for k in range(self.K):
            self.Pz[k] = np.sum(self.p[i, j, k] * X[i, j] for i in range(N) for j in range(M))

        # Chuẩn hóa Pz: chia cho tổng số lần xuất hiện của tất cả các từ và tài liệu
        total = np.sum(X)
        self.Pz /= total


        
    def LogLikelihood(self, N, M, X):
        loglikelihood = 0
        for i in range(0, N):
            for j in range(0, M):
                tmp = 0
                for k in range(0, self.K):
                    tmp += self.theta[j,k] * self.lamda[i, k] * self.Pz[k]
                if tmp > 0:
                    loglikelihood += X[i, j] * log(tmp)
        return loglikelihood

    def train(self, X, M, N):
        # N, M, X = self.preprocessing(dataset)
        # self.initializeParameters(N, M)

        oldLoglikelihood = 1
        newLoglikelihood = 1
        self.p = zeros((N,M,self.K))
        for i in range(0, 1):
            self.EStep(N, M)
            self.MStep(N, M, X)
            # print(self.Pz)
            newLoglikelihood = self.LogLikelihood(N, M, X)
            # if oldLoglikelihood != 1 and newLoglikelihood - oldLoglikelihood < self.threshold:
            #     break
            oldLoglikelihood = newLoglikelihood
            # print(newLoglikelihood)


            print("Step: ", i)
            print("Pz\n", self.Pz)
            print("theta\n", self.theta)
            print("lamda\n", self.lamda)





In [30]:
x = [[1, 2, 0, 0, 0, 0],
     [3, 1, 0, 0, 0, 0],
     [2, 0, 0, 0, 0, 0],
     [3, 3, 2, 3, 2, 4],
     [0, 0, 3, 2, 0, 0],
     [0, 0, 4, 1, 0, 0],
     [0, 0, 0, 0, 4, 3],
     [0, 0, 0, 0, 2, 1],
     [0, 0, 0, 0, 3, 2],
     [0, 0, 1, 0, 2, 3]]
x = np.array(x)
n = 10
m = 6
lamda = [[0.022, 0.016, 0.010],
          [0.018, 0.133, 0.166],
          [0.242, 0.058, 0.133],
          [0.123, 0.088, 0.145],
          [0.016, 0.030, 0.044],
          [0.020, 0.167, 0.056],
          [0.147, 0.129, 0.201],
          [0.188, 0.156, 0.039],
          [0.146, 0.114, 0.008],
          [0.077, 0.110, 0.199]]

theta = [[0.020, 0.008, 0.048],
        [0.294, 0.255, 0.329],
        [0.204, 0.138, 0.178],
        [0.200, 0.146, 0.007],
        [0.186, 0.196, 0.233],
        [0.096, 0.257, 0.205]]

Pz = [0.525, 0.407, 0.068]
Pz = np.array(Pz)
theta = np.array(theta)
lamda = np.array(lamda)
# k = 3
model = PLSA(K=3, maxIteration=70,threshold=5.0, topicWordsNum=4)

In [31]:
model.initializeParameters(n,m,Pz,lamda, theta)
model.train(x,m,n)

Initialization
Pz
 [0.525 0.407 0.068]
theta
 [[0.02  0.008 0.048]
 [0.294 0.255 0.329]
 [0.204 0.138 0.178]
 [0.2   0.146 0.007]
 [0.186 0.196 0.233]
 [0.096 0.257 0.205]]
lamda
 [[0.022 0.016 0.01 ]
 [0.018 0.133 0.166]
 [0.242 0.058 0.133]
 [0.123 0.088 0.145]
 [0.016 0.03  0.044]
 [0.02  0.167 0.056]
 [0.147 0.129 0.201]
 [0.188 0.156 0.039]
 [0.146 0.114 0.008]
 [0.077 0.11  0.199]]
Step:  0
Pz
 [0.45821703 0.42984328 0.11193969]
theta
 [[0.18047836 0.07426782 0.38657384]
 [0.12401997 0.08908869 0.09059273]
 [0.14647945 0.21328815 0.14864023]
 [0.12504987 0.11053789 0.00401299]
 [0.26621435 0.20382629 0.16502518]
 [0.15775801 0.30899117 0.20515504]]
lamda
 [[0.07725898 0.03240679 0.02948333]
 [0.02389351 0.07331716 0.24756306]
 [0.06149752 0.00487221 0.04300727]
 [0.37166842 0.22132574 0.29306347]
 [0.08699721 0.09425399 0.0655822 ]
 [0.03356232 0.15913385 0.03517836]
 [0.11489448 0.13006763 0.12731601]
 [0.05851531 0.05761792 0.00939963]
 [0.09880349 0.09761685 0.00434082]
 [0.07

C:\Users\Admin\AppData\Local\Temp\ipykernel_7080\139751936.py:36: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  denominator = np.sum(self.Pz[k] * self.lamda[i,k] * self.theta[j,k] for k in range(self.K))
C:\Users\Admin\AppData\Local\Temp\ipykernel_7080\139751936.py:48: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  self.theta[j,k] = np.sum(X[i, j] * self.p[i, j, k] for i in range(N))
C:\Users\Admin\AppData\Local\Temp\ipykernel_7080\139751936.py:54: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  self.lamda[i, k] = np.sum(X[i, j] * self.p[i, j, k] for j in range(M))
C:\Users\Admin\AppData\Loc

In [1]:
import re
import codecs
import numpy as np
import pickle
from numpy import zeros, int8, log
from pyvi import ViTokenizer
from pylab import random




class PLSA:
    def __init__(self, K, maxIteration=30, threshold=10.0, topicWordsNum=5):
        self.K = K  # Số chủ đề
        self.maxIteration = maxIteration  # Số lần lặp tối đa
        self.threshold = threshold  # Ngưỡng dừng
        self.topicWordsNum = topicWordsNum  # Số từ hàng đầu trong mỗi chủ đề
        self.word2id = {}  
        self.id2word = {}  
        self.Pz = zeros(self.K)  # Xác suất của các chủ đề
        self.lamda = None  # Ma trận lambda P(d|z)
        self.theta = None  # Ma trận theta P(w|z)
        self.p = None  # Ma trận xác suất trung gian P(z|d,w)
        self.stopwordsFilePath='stopwords.dic'  
    
    def preprocessing(self, dataset):
        """
        Xử lý trước tài liệu, bao gồm tokenization, loại bỏ stopwords và xây dựng
        word2id, id2word và ma trận tần suất từ.
        """
        # Load stopwords
        file = codecs.open(self.stopwordsFilePath, 'r', 'utf-8')
        self.stopwords = [line.lower().strip() for line in file]
        file.close()

        documents = dataset
        N = len(documents)  # Số tài liệu
        wordCounts = []
        currentId = 0

        # Xây dựng word2id và id2word, đếm số lượng từ
        for document in documents:
            segList = ViTokenizer.tokenize(document).split()
            wordCount = {}
            for word in segList:
                word = word.lower().strip()
                if len(word) > 1 and not re.search('[0-9]', word) and word not in self.stopwords:
                    if word not in self.word2id:
                        self.word2id[word] = currentId
                        self.id2word[currentId] = word
                        currentId += 1
                    if word in wordCount:
                        wordCount[word] += 1
                    else:
                        wordCount[word] = 1
            wordCounts.append(wordCount)

        M = len(self.word2id)  # Số từ
        X = zeros([N, M], int8)
        for word in self.word2id.keys():
            j = self.word2id[word]
            for i in range(0, N):
                if word in wordCounts[i]:
                    X[i, j] = wordCounts[i][word]

        return N, M, X

    def initializeParameters(self, N, M):
        self.lamda = random([N, self.K])  # P(d|z)
        self.theta = random([M, self.K])  # P(w|z)
        self.Pz = random([self.K])  # P(z)
        self.Pz /= sum(self.Pz)  # Chuẩn hóa Pz để tổng bằng 1

        self.p = np.zeros((N, M, self.K))  # Ma trận xác suất P(z|d,w)

        # Chuẩn hóa lambda và theta
        for k in range(0, self.K):
            self.lamda[:, k] /= sum(self.lamda[:, k])
            self.theta[:, k] /= sum(self.theta[:, k])
            

    def EStep(self, N, M, X):
        """
        Thực hiện bước E: Tính toán ma trận xác suất P(z|d,w).
        """
        for i in range(0, N):
            for j in range(0, M):
                denominator = np.sum(self.Pz[k] * self.lamda[i,k] * self.theta[j,k] for k in range(self.K))
                if denominator == 0:
                    self.p[i,j,:] = 1.0/self.K
                else:
                    for k in range(self.K):
                        self.p[i,j,k] = (self.Pz[k] * self.lamda[i,k] * self.theta[j,k]) / denominator    

    def MStep(self, N, M, X):
        # Cập nhật theta (P(w|z))
        for k in range(0, self.K):
            for j in range(0, M):
                self.theta[j,k] = np.sum(X[i,j] * self.p[i,j,k] for i in range(N))
            self.theta[:, k] /= np.sum(self.theta[:, k])

        # Cập nhật lambda (P(d|z))
        for k in range(self.K):
            for i in range(N):
                self.lamda[i,k] = np.sum(X[i,j] * self.p[i,j,k] for j in range(M))
            self.lamda[:, k] /= np.sum(self.lamda[:, k])

        for k in range(self.K):
            self.Pz[k] = np.sum(self.p[i,j,k] * X[i,j] for i in range(N) for j in range(M))

        # Chuẩn hóa Pz: chia cho tổng số lần xuất hiện của tất cả các từ và tài liệu
        total = np.sum(X)
        self.Pz /= total


        
    def LogLikelihood(self, N, M, X):
        loglikelihood = 0
        for i in range(0, N):
            for j in range(0, M):
                tmp = 0
                for k in range(0, self.K):
                    tmp += self.theta[j, k] * self.lamda[i, k] * self.Pz[k]
                if tmp > 0:
                    loglikelihood += X[i, j] * log(tmp)
        return loglikelihood

    def train(self, dataset):
        N, M, X = self.preprocessing(dataset)
        self.initializeParameters(N, M)

        oldLoglikelihood = 1
        newLoglikelihood = 1
        # print("training")
        for i in range(0, self.maxIteration):
            self.EStep(N, M, X)
            self.MStep(N, M, X)
            # print(self.Pz)
            newLoglikelihood = self.LogLikelihood(N, M, X)
            if oldLoglikelihood != 1 and newLoglikelihood - oldLoglikelihood < self.threshold:
                break
            oldLoglikelihood = newLoglikelihood
            # print(newLoglikelihood)

        topic = self.get_top_words()
        return self.p, self.Pz, self.lamda, self.theta, topic, self.id2word

    def get_top_words(self):
        topic = []
        for i in range(0, self.K):
            topicword = []
            ids = self.theta[:, i].argsort()
            for j in ids:
                topicword.insert(0, self.id2word[j])
            tmp = ''
            for word in topicword[0:min(self.topicWordsNum, len(topicword))]:
                tmp += word + ' '
            topic.append(tmp)
        return topic


    def test(self, data_test):
        segList= ViTokenizer.tokenize(data_test[0])    
        xtest = zeros(len(self.id2word))
        for word in segList.split(' '):
            word = word.lower().strip()
            # print(word)
            for i in range(len(self.id2word)):
                if word == self.id2word[i]:
                    xtest[i] += 1

        z = []
        for k in range(self.K):
            p_new = 1
            for i in range(len(xtest)):
                if xtest[i] != 0:
                    # if self.theta[k,i] > 1e-6:
                        p_new *= self.theta[i, k] * xtest[i]
                        # print(p_new)
            if p_new == 1:
                z.append(0)
            else:
                p_new *= self.Pz[k]
                z.append(p_new)
        # print(z)

        if all(i == 0 for i in z):
            return "Khong thuoc chu de nao trong cac chu de tren"  
        else:
            main_topic = z.index(max(z))
            return 'Pz'+str(main_topic)



    def save_model(self, filename):
        with open(filename, 'wb') as f:
            pickle.dump({
                'Pz': self.Pz,
                'theta': self.theta,
                'lamda': self.lamda,
                'id2word': self.id2word,
                'word2id': self.word2id,
            }, f)

    def load_model(self, filename):
        with open(filename, 'rb') as f:
            params = pickle.load(f)
            self.Pz = params['Pz']
            self.theta = params['theta']
            self.lamda = params['lamda']
            self.id2word = params['id2word']
            self.word2id = params['word2id']
        topic = self.get_top_words()
        self.K = len(self.Pz)
        return self.p, self.Pz, self.lamda, self.theta, topic, self.id2word

In [2]:
file = codecs.open('data_train.txt', 'r', 'utf-8')
train_data = [document.strip() for document in file] 
file.close()


model = PLSA(K=3, maxIteration=70,threshold=5.0, topicWordsNum=4)

# m,n,x = model.preprocessing(dataset=train_data)
p, Pz, lamda, theta, wordTop, id2w = model.train(dataset=train_data)

C:\Users\Admin\AppData\Local\Temp\ipykernel_4368\1933223508.py:88: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  denominator = np.sum(self.Pz[k] * self.lamda[i,k] * self.theta[j,k] for k in range(self.K))
C:\Users\Admin\AppData\Local\Temp\ipykernel_4368\1933223508.py:99: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  self.theta[j,k] = np.sum(X[i,j] * self.p[i,j,k] for i in range(N))
C:\Users\Admin\AppData\Local\Temp\ipykernel_4368\1933223508.py:105: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  self.lamda[i,k] = np.sum(X[i,j] * self.p[i,j,k] for j in range(M))
C:\Users\Admin\AppData\Local\

In [3]:
print("Pz", Pz)
print(wordTop)

Pz [0.29898097 0.43451343 0.2665056 ]
['đại_học việt_nam phát_triển trường ', 'nước theo quán ăn ', 'trường thi điểm học_sinh ']


In [8]:
np.sum(p)

74704.0